In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
restViol = pd.read_csv('restViol.csv', parse_dates=['INSPECTION DATE'])
dataPois = pd.read_csv('dataPois.csv', parse_dates=['Created Date'])
data311comp = pd.read_csv('data311Pois.csv', parse_dates=['Created Date'])

/Users/devashishkhulbe/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ct = gpd.read_file('/Users/devashishkhulbe/Downloads/Data/2010 Census Tracts/geo_export_a0d9c6b6-aa01-4614-864e-ae8521d74700.shp')
pop = pd.read_csv('/Users/devashishkhulbe/Downloads/Data/New_York_City_Population_By_Census_Tracts.csv')
ct = ct[['boro_code', 'boro_ct201', 'boro_name', 'ct2010', 'geometry', 'shape_area']]
ct['ct2010'] = ct['ct2010'].astype(int)
pop = pop[pop.Year == 2010]
ct['bct'] = ct['boro_code'].astype(str) + ct['ct2010'].astype(str)
pop['bct'] = pop['DCP Borough Code'].astype(str) + pop['Census Tract'].astype(str)
pop = pop[['bct', 'Year', 'Borough', 'Population']]
ct = ct.merge(pop, on='bct')
ct.head()

,boro_code,boro_ct201,boro_name,ct2010,geometry,shape_area,bct,Year,Borough,Population
0,5,5000900,Staten Island,900,POLYGON ((-74.07920577013245 40.64343078374567...,2.497010e+06,5900,2010,Staten Island,2112
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316
2,1,1010000,Manhattan,10000,POLYGON ((-73.96802436915851 40.75957814005282...,1.860938e+06,110000,2010,Manhattan,1992
3,1,1010200,Manhattan,10200,POLYGON ((-73.97124277307127 40.76093641847906...,1.860993e+06,110200,2010,Manhattan,230
4,1,1010400,Manhattan,10400,POLYGON ((-73.97445730550224 40.76229308352487...,1.864600e+06,110400,2010,Manhattan,966


In [4]:
geometry = [Point(xy) for xy in zip(restViol.lng, restViol.lat)]
crs = {'init': 'epsg:4326'}
restViol = gpd.GeoDataFrame(restViol, crs=crs, geometry=geometry)

geometry = [Point(xy) for xy in zip(dataPois.Longitude, dataPois.Latitude)]
crs = {'init': 'epsg:4326'}
dataPois = gpd.GeoDataFrame(dataPois, crs=crs, geometry=geometry)

geometry = [Point(xy) for xy in zip(data311comp.Longitude, data311comp.Latitude)]
crs = {'init': 'epsg:4326'}
data311comp = gpd.GeoDataFrame(data311comp, crs=crs, geometry=geometry)

In [5]:
restViol.drop(columns=['Unnamed: 0', 'RECORD DATE', 'INSPECTION TYPE', 'PHONE'], inplace=True)
dataPois = dataPois[['Unique Key', 'Created Date', 'Closed Date', 'Complaint Type', 'Descriptor', 
                    'Incident Zip', 'Incident Address', 'Latitude', 'Longitude', 'geometry']]
data311comp = data311comp[['Unique Key', 'Created Date', 'Closed Date', 'Complaint Type', 'Descriptor', 
                    'Incident Zip', 'Incident Address', 'Latitude', 'Longitude', 'geometry']]

In [6]:
restViols = gpd.sjoin(ct, restViol)
dataPoisn = gpd.sjoin(ct, dataPois)
data311comps = gpd.sjoin(ct, data311comp)

/Users/devashishkhulbe/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in find_intersects (vectorized)
  outputs = ufunc(*inputs)


In [38]:
restViolNT = restViols.groupby(by='boro_ct201', as_index=False).count()[['boro_ct201', 'DBA']]
restViolNT = ct.merge(restViolNT, on='boro_ct201', how='outer')
restViolNT.fillna(value={'DBA':0}, inplace=True)
restViolNT.rename(columns={'DBA':'rest_Violations'}, inplace=True)
restViolNT.to_csv('restViolNT.csv')

In [42]:
dataPoisNT = dataPoisn.groupby(by='boro_ct201', as_index=False).count()[['boro_ct201', 'Created Date']]
dataPoisNT = ct.merge(dataPoisNT, on='boro_ct201', how='outer')
dataPoisNT.fillna(value={'Created Date':0}, inplace=True)
dataPoisNT.rename(columns={'Created Date':'pois_complaints'}, inplace=True)
dataPoisNT.to_csv('dataPoisNT.csv')

In [46]:
data311compNT = data311comps.groupby(by='boro_ct201', as_index=False).count()[['boro_ct201', 'Created Date']]
data311compNT = ct.merge(data311compNT, on='boro_ct201', how='outer')
data311compNT.fillna(value={'Created Date':0}, inplace=True)
data311compNT.rename(columns={'Created Date':'pois_complaints'}, inplace=True)
data311compNT.to_csv('data311compNT.csv')

In [84]:
temp35 = restViols.groupby(by=['boro_ct201', 'INSPECTION DATE'], as_index=False).count()
temp36 = temp35.boro_ct201.unique()

frames = []

for boro in temp36:
    temp = temp35[temp35.boro_ct201 == boro]
    #print temp
    temp1 = temp.groupby(pd.Grouper(key='INSPECTION DATE', freq='M')).sum()[['DBA']]
    #print temp1
    temp1['boro_ct201'] = boro
    temp1.reset_index(inplace=True)
    frames.append(temp1)
    
    print boro

1000201
1000202
1000500
1000600
1000700
1000800
1000900
1001200
1001300
1001401
1001402
1001501
1001502
1001600
1001800
1002100
1002201
1002202
1002500
1002601
1002602
1002700
1002800
1002900
1003001
1003002
1003100
1003200
1003300
1003400
1003601
1003602
1003700
1003800
1003900
1004000
1004100
1004200
1004300
1004400
1004500
1004700
1004800
1004900
1005000
1005200
1005400
1005501
1005502
1005600
1005700
1005800
1005900
1006000
1006100
1006200
1006300
1006400
1006500
1006600
1006700
1006800
1006900
1007000
1007100
1007200
1007300
1007400
1007500
1007600
1007700
1007800
1007900
1008000
1008100
1008200
1008300
1008400
1008601
1008603
1008700
1008800
1008900
1009000
1009100
1009200
1009300
1009400
1009500
1009600
1009700
1009800
1009900
1010000
1010100
1010200
1010300
1010400
1010601
1010602
1010800
1010900
1011000
1011100
1011201
1011202
1011203
1011300
1011401
1011402
1011500
1011600
1011700
1011800
1011900
1012000
1012100
1012200
1012400
1012500
1012600
1012700
1012800
1012900
1013000


3059200
3059300
3059401
3059402
3059600
3060000
3060600
3060800
3061003
3061004
3061600
3062000
3062200
3062600
3062800
3063200
3063800
3064200
3064400
3064600
3064800
3065000
3065200
3065600
3065800
3066200
3066600
3067000
3067200
3067400
3067600
3068000
3068200
3068800
3069000
3069601
3069602
3069800
3070000
3070201
3070202
3070600
3072000
3072200
3072400
3072600
3072800
3073000
3073400
3073600
3073800
3074000
3074200
3074600
3074800
3075000
3076000
3076800
3077000
3077200
3077400
3077600
3078000
3078600
3078800
3079000
3079200
3079400
3079601
3079602
3079801
3079802
3080000
3080200
3080400
3080600
3080800
3081000
3081400
3081600
3081800
3082000
3082200
3082400
3082600
3082800
3083000
3083200
3083400
3083600
3084000
3084600
3084800
3085600
3085800
3086000
3086200
3086400
3086600
3086800
3087000
3087200
3087401
3087600
3087800
3088000
3088200
3088400
3088600
3088800
3089000
3089200
3089400
3089600
3089800
3090000
3090200
3090600
3090800
3091000
3091600
3091800
3092000
3092200
3092400


In [71]:
restViols

,boro_code,boro_ct201,boro_name,ct2010,geometry,shape_area,bct,Year,Borough,Population,...,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,Address,lat,lng
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,11.0,NaN,NaN,"PARNELL'S PUB, 350 EAST 53 STREET, MANHATTAN...",40.755745,-73.965015
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,26.0,B,08/01/2016,"PARNELL'S PUB, 350 EAST 53 STREET, MANHATTAN...",40.755745,-73.965015
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,26.0,B,08/01/2016,"PARNELL'S PUB, 350 EAST 53 STREET, MANHATTAN...",40.755745,-73.965015
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,8.0,NaN,NaN,"PARNELL'S PUB, 350 EAST 53 STREET, MANHATTAN...",40.755745,-73.965015
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,19.0,B,08/02/2018,"PARNELL'S PUB, 350 EAST 53 STREET, MANHATTAN...",40.755745,-73.965015
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,8.0,A,11/24/2015,"THE IRISH EXIT, 978 2 AVENUE, MANHATTAN, 10022",40.755834,-73.967601
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,12.0,A,09/06/2018,"THE IRISH EXIT, 978 2 AVENUE, MANHATTAN, 10022",40.755834,-73.967601
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,12.0,NaN,NaN,"THE IRISH EXIT, 978 2 AVENUE, MANHATTAN, 10022",40.755834,-73.967601
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,10.0,NaN,NaN,"THE IRISH EXIT, 978 2 AVENUE, MANHATTAN, 10022",40.755834,-73.967601
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,...,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,NaN,NaN,"THE IRISH EXIT, 978 2 AVENUE, MANHATTAN, 10022",40.755834,-73.967601


In [88]:
restViolT = pd.concat(frames)
restViolT.rename(columns={'DBA':'restViolations'}, inplace=True)
restViolT.to_csv('restViolT.csv')

In [98]:
dataPoisn.head()

,boro_code,boro_ct201,boro_name,ct2010,geometry,shape_area,bct,Year,Borough,Population,index_right,Unique Key,Created Date,Closed Date,Complaint Type,Descriptor,Incident Zip,Incident Address,Latitude,Longitude
0,5,5000900,Staten Island,900,POLYGON ((-74.07920577013245 40.64343078374567...,2.497010e+06,5900,2010,Staten Island,2112,10713,29304896,2014-11-16,NaN,Food Poisoning,3 or More,10301.0,19 FORT PLACE,40.641972,-74.079021
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,1615,35593436,2017-02-28,03/01/2017 12:24:43 PM,Food Poisoning,1 or 2,10022.0,1005 2 AVENUE,40.756896,-73.967070
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,12480,30887360,2015-06-19,NaN,Food Poisoning,1 or 2,10022.0,1008 2 AVENUE,40.756948,-73.967005
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,259,34362839,2016-09-20,09/30/2016 12:00:01 AM,Food Poisoning,1 or 2,10022.0,251 EAST 53 STREET,40.757234,-73.968254
1,1,1009800,Manhattan,9800,POLYGON ((-73.96432543478758 40.75638153099091...,1.906016e+06,19800,2010,Manhattan,7316,5243,37531174,2017-10-26,10/27/2017 09:12:15 AM,Food Poisoning,1 or 2,10022.0,231 EAST 53 STREET,40.757366,-73.968572


In [103]:
temp35 = dataPoisn.groupby(by=['boro_ct201', 'Created Date'], as_index=False).count()
temp36 = temp35.boro_ct201.unique()

frames = []

for boro in temp36:
    temp = temp35[temp35.boro_ct201 == boro]
    temp.index = pd.DatetimeIndex(temp['Created Date'])
    temp1 = temp.groupby(pd.Grouper(freq='M')).sum()[['Complaint Type']]
    #print temp1
    temp1['boro_ct201'] = boro
    temp1.reset_index(inplace=True)
    frames.append(temp1)
    
    print boro

1000100
1000201
1000202
1000600
1000700
1000800
1000900
1001200
1001300
1001401
1001402
1001501
1001502
1001600
1001800
1002000
1002100
1002201
1002202
1002500
1002601
1002602
1002700
1002800
1002900
1003001
1003002
1003100
1003200
1003300
1003400
1003601
1003602
1003700
1003800
1003900
1004000
1004100
1004200
1004300
1004400
1004500
1004700
1004800
1004900
1005000
1005200
1005400
1005501
1005502
1005600
1005700
1005800
1005900
1006000
1006100
1006200
1006300
1006400
1006500
1006600
1006700
1006800
1006900
1007000
1007100
1007200
1007300
1007400
1007500
1007600
1007700
1007800
1007900
1008000
1008100
1008200
1008300
1008400
1008700
1008800
1008900
1009000
1009100
1009200
1009300
1009400
1009500
1009600
1009700
1009800
1009900
1010000
1010100
1010200
1010300
1010400
1010601
1010602
1010800
1010900
1011000
1011100
1011201
1011202
1011203
1011300
1011401
1011402
1011500
1011600
1011700
1011800
1011900
1012000
1012100
1012200
1012400
1012500
1012600
1012700
1012800
1012900
1013000
1013100


3067600
3068000
3068800
3069000
3069601
3069602
3069800
3070000
3070201
3072000
3072200
3072400
3072800
3073600
3073800
3074000
3074200
3074400
3074600
3074800
3075000
3076000
3076800
3077000
3077400
3077600
3078000
3078600
3078800
3079000
3079200
3079400
3079601
3079602
3079801
3079802
3080000
3080200
3080400
3080600
3080800
3081600
3081800
3082000
3082200
3082400
3082600
3082800
3083000
3083400
3083600
3084000
3084600
3084800
3085400
3085600
3085800
3086000
3086200
3086400
3086600
3086800
3087000
3087200
3087401
3087600
3087800
3088000
3088200
3088400
3088600
3088800
3089000
3089200
3089400
3089600
3089800
3090000
3090200
3090600
3090800
3091000
3091200
3091600
3091800
3092000
3092200
3092400
3092800
3093000
3093400
3094401
3094402
3094600
3095000
3095400
3095600
3095800
3096000
3096200
3096400
3096600
3096800
3097400
3098200
3098400
3099000
3099400
3099600
3101000
3101200
3101800
3102200
3102400
3102800
3103400
3105804
3107000
3109800
3110400
3111600
3112000
3112200
3112400
3112600


In [108]:
dataPoisT = pd.concat(frames)
dataPoisT.rename(columns={'Complaint Type':'poisCases'}, inplace=True)
dataPoisT.to_csv('dataPoisT.csv')

In [109]:
temp35 = data311comps.groupby(by=['boro_ct201', 'Created Date'], as_index=False).count()
temp36 = temp35.boro_ct201.unique()

frames = []

for boro in temp36:
    temp = temp35[temp35.boro_ct201 == boro]
    temp.index = pd.DatetimeIndex(temp['Created Date'])
    temp1 = temp.groupby(pd.Grouper(freq='M')).sum()[['Complaint Type']]
    #print temp1
    temp1['boro_ct201'] = boro
    temp1.reset_index(inplace=True)
    frames.append(temp1)
    
    print boro

1000100
1000201
1000202
1000600
1000700
1000800
1000900
1001200
1001300
1001401
1001402
1001501
1001502
1001600
1001800
1002000
1002100
1002201
1002202
1002400
1002500
1002601
1002602
1002700
1002800
1002900
1003001
1003002
1003100
1003200
1003300
1003400
1003601
1003602
1003700
1003800
1003900
1004000
1004100
1004200
1004300
1004400
1004500
1004700
1004800
1004900
1005000
1005200
1005400
1005501
1005502
1005600
1005700
1005800
1005900
1006000
1006100
1006200
1006300
1006400
1006500
1006600
1006700
1006800
1006900
1007000
1007100
1007200
1007300
1007400
1007500
1007600
1007700
1007800
1007900
1008000
1008100
1008200
1008300
1008400
1008601
1008602
1008603
1008700
1008800
1008900
1009000
1009100
1009200
1009300
1009400
1009500
1009600
1009700
1009800
1009900
1010000
1010100
1010200
1010300
1010400
1010601
1010602
1010800
1010900
1011000
1011100
1011201
1011202
1011203
1011300
1011401
1011402
1011500
1011600
1011700
1011800
1011900
1012000
1012100
1012200
1012400
1012500
1012600
1012700


3053200
3053300
3053400
3053500
3053700
3053800
3054200
3054300
3054500
3054600
3054700
3054900
3055000
3055100
3055200
3055300
3055400
3055500
3055600
3055700
3055800
3056100
3056300
3056400
3056500
3056600
3056900
3057000
3057100
3057200
3057300
3057400
3057500
3057600
3057800
3057900
3058000
3058200
3058400
3058600
3058800
3058900
3059000
3059100
3059200
3059300
3059401
3059402
3059600
3060000
3060600
3060800
3061002
3061003
3061004
3061600
3062000
3062200
3062600
3062800
3064000
3064200
3064400
3064600
3064800
3065000
3065200
3065600
3066200
3066600
3067000
3067200
3067400
3067600
3067800
3068000
3068200
3068600
3068800
3069000
3069601
3069602
3069800
3070201
3072000
3072200
3072400
3072600
3072800
3073000
3073600
3073800
3074000
3074200
3074600
3074800
3075000
3075600
3076400
3076600
3076800
3077000
3077200
3077400
3077600
3078000
3078400
3078600
3078800
3079000
3079200
3079400
3079601
3079602
3079801
3079802
3080000
3080200
3080400
3080600
3080800
3081000
3081400
3081600
3081800


In [114]:
data311compT = pd.concat(frames)
data311compT.rename(columns={'Complaint Type':'311comp'}, inplace=True)
data311compT.to_csv('data311compT.csv')

In [116]:
ManRest = pd.read_csv('Manhattan.csv')
ManRest = ManRest[['restaurant_alias', 'restaurant_coordinates_latitude', 'restaurant_coordinates_longitude']]
BrookRest = pd.read_csv('Brooklyn.csv')
BrookRest = BrookRest[['restaurant_alias', 'restaurant_coordinates_latitude', 'restaurant_coordinates_longitude']]
QueenRest = pd.read_csv('Queens.csv')
QueenRest = QueenRest[['restaurant_alias', 'restaurant_coordinates_latitude', 'restaurant_coordinates_longitude']]
BronxRest = pd.read_csv('Bronx.csv')
BronxRest = BronxRest[['restaurant_alias', 'restaurant_coordinates_latitude', 'restaurant_coordinates_longitude']]
SIRest = pd.read_csv('StatenIsland.csv')
SIRest = SIRest[['restaurant_alias', 'restaurant_coordinates_latitude', 'restaurant_coordinates_longitude']]

frames = [ManRest, BrookRest, QueenRest, BronxRest, SIRest]
AllRest = pd.concat(frames)

AllRest.head()

,restaurant_alias,restaurant_coordinates_latitude,restaurant_coordinates_longitude
0,barn-joo-35-new-york,40.749640,-73.985820
1,amélie-new-york,40.732700,-73.997660
2,upstate-new-york-2,40.726314,-73.986490
3,bea-new-york,40.759197,-73.992336
4,ipanema-restaurant-new-york,40.757072,-73.980780


In [117]:
AllRest.shape

(4500, 3)

In [118]:
geometry = [Point(xy) for xy in zip(AllRest.restaurant_coordinates_longitude, AllRest.restaurant_coordinates_latitude)]
crs = {'init': 'epsg:4326'}
AllRest = gpd.GeoDataFrame(AllRest, crs=crs, geometry=geometry)
AllRest.head()

,restaurant_alias,restaurant_coordinates_latitude,restaurant_coordinates_longitude,geometry
0,barn-joo-35-new-york,40.749640,-73.985820,POINT (-73.98581999999999 40.74964)
1,amélie-new-york,40.732700,-73.997660,POINT (-73.99766 40.7327)
2,upstate-new-york-2,40.726314,-73.986490,POINT (-73.98649019770001 40.7263139141)
3,bea-new-york,40.759197,-73.992336,POINT (-73.99233609999999 40.7591968)
4,ipanema-restaurant-new-york,40.757072,-73.980780,POINT (-73.9807795529 40.7570722173)


In [124]:
Rests = gpd.sjoin(ct, AllRest)
Rests = Rests.groupby(by='boro_ct201', as_index=False).count()[['boro_ct201', 'restaurant_alias']]
Rests.rename(columns={'restaurant_alias':'Nrestaurants'}, inplace=True)
Rests.head()

,boro_ct201,Nrestaurants
0,1000500,1
1,1000600,7
2,1000700,5
3,1000800,1
4,1000900,10


In [125]:
Rests.to_csv('Rests.csv')

In [25]:
restViols['boro_ct201'] = restViols['boro_ct201'].astype(int)
boros = restViols['boro_ct201'].unique()
NrestsWviol = []
#boros
for boro in boros:
    temp = restViols[restViols.boro_ct201 == boro]
    NrestsWviol.append(len(temp['Address'].unique()))
    
    print boro

1009800
1010000
1010200
1010400
1011300
1011402
1013000
1014000
1014801
1018400
1019000
1024900
1026100
1026300
1026700
1026900
1027700
3000501
3002000
3004100
3005800
3006000
3006700
3006900
3007400
3007600
3007800
3008000
3009800
3010100
3010200
3010400
3010800
3011000
3011400
3012801
3013000
3013200
3013300
3013600
3013700
3020800
3013900
3015100
3015300
3015500
3015700
3017000
3017800
3018800
3019000
3019200
3019400
3019500
3019600
3019700
3019800
3021000
3021200
3021400
3021900
3022000
3022100
3022200
3022700
3023500
3023600
3023800
3024000
3024200
3024500
3024600
3024700
3024800
3024900
3025000
3025200
3025400
3025600
3025800
3026000
3026200
3026300
3026400
3026600
3026700
3026800
3026900
3027300
3027400
3027600
3027800
3027900
3028400
3028501
3028800
3029100
3029700
3029800
3030000
3030500
3042600
3030900
3031701
3032300
3032900
3033100
3033700
3033900
3034100
3034200
3036300
3038800
3039700
3039900
3040100
3040400
3040800
3041401
3041500
3041700
3041800
3041900
3042000
3042100


2024800
2029600
4032800
4047600
4047800
2035000
4029500
4029700
5001100
5001700
4152901
4065000
3070600
2026300
2032400
2032600
3015400
3030400
3018200
3038200
3040200
5014100
5007700
4009900
5010500
5023900
5024700
2020000
2028600
2020502
2040702
3088400
2046000
5002900
5003300
4010100
4009700
5013302
5018702
5020804
3087200
2035100
2038000
2033800
3116600
3116800
3042500
3038900
3039100
1015100
1013600
3003300
3072400
3110400
3112200
1010900
4005200
4005400
3111600
3112000
3095600
3095800
3099000
1022900
3118202
1023300
1022000
3101200
3027100
3081600
1003100
1016400
3090600
1018000
1018200
4089200
3120800
1002201
1002202
3049100
3050700
1020901
3050900
3089200
3089400
3090000
1005800
1008900
1009300
1011100
1022301
4004401
3037100
4055900
3037900
2043100
3033500
3035300
3029900
3038100
4003400
3118600
3035500
3088200
2033600
2034800
2036400
1019300
1007500
4088400
3066200
4063900
3069800
3101800
3105804
4023500
4056700
4137700
4138502
3040500
5019700
3085600
3085800
3086000
2041400


In [26]:
len(NrestsWviol)

1871

In [33]:
NrestWviol = pd.DataFrame(boros, NrestsWviol)
NrestWviol.reset_index(inplace=True)
NrestWviol.rename(columns={'index':'NrestWviol', 0:'boro_ct201'}, inplace=True)
NrestWviol.to_csv('NrestWviol.csv')

In [47]:
#restViols.replace(np.nan, '', regex=True, inplace=True)
restViols['GRADE'] = restViols['GRADE'].astype(str)
def grades(grade):
    NewGr = []
    
    for gr in grade:
        if gr == 'A':
            NewGr.append(3)
        elif gr == 'B':
            NewGr.append(2)
        elif gr == 'C':
            NewGr.append(1)
        else:
            NewGr.append(np.nan)

NewGrades = grades(restViols['GRADE'].values)

In [1]:
NewGrades

NameError: name 'NewGrades' is not defined